In [14]:
from __future__ import division, print_function, absolute_import

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

In [42]:
#Xavier Glorot init
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0]/2.))

def readivs(sourceD):
    vals = []
    indices = []
    sourceFList = [one for one in os.listdir(sourceD) if one.endswith(".ark") and one.startswith("ivector")]
    for sourceF in sourceFList:
        with open(os.path.join(sourceD,sourceF)) as s:
            for line in s:
                step1 = line.split("[")
                index = step1[0]
                indices.append(index)
                content = step1[1].split("]")[0]
                values = np.array(map(float, content.split()))
                vals.append(values)
    #index is seperated by -, speakerid-file(channelid)
    return indices, np.array(vals)

sourceD = "/work/jiacen/exp/ivectors_train_female"
indices, ivs = readivs(sourceD)
global batch_start, dataset_size
batch_start = 0
dataset_size = ivs.shape[0]

def next_batch(batch_size):
    global batch_start, dataset_size
    if batch_start + batch_size > dataset_size:
        result = np.append(ivs[batch_start:], ivs[:(batch_start+batch_size)%dataset_size],axis=0)
        batch_start = (batch_start+batch_size)%dataset_size
        return result
    result = ivs[batch_start: batch_size + batch_start]
    batch_start += batch_size
    return result
        
    

In [31]:
#Training Params
num_steps = 100000
batch_size = 128
learning_rate = 0.0002

#Network Params
input_dim = 400
gen_hidden_dim = 256
disc_hidden_dim = 256
noise_dim = 100

#Layers weight & bias
weights = {
    'gen_hidden1': tf.Variable(glorot_init([noise_dim, gen_hidden_dim])),
    'gen_out': tf.Variable(glorot_init([gen_hidden_dim, input_dim])),
    'disc_hidden1': tf.Variable(glorot_init([input_dim, disc_hidden_dim])),
    'disc_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([input_dim])),
    'disc_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc_out': tf.Variable(tf.zeros([1])),
}

In [32]:
#Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

#Discriminator
def discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc_out'])
    out_layer = tf.add(out_layer, biases['disc_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

#Build Networks

#Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, input_dim], name='disc_input')

#Generator Network
gen_sample = generator(gen_input)

disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample)

#Build loss
gen_loss = -tf.reduce_mean(tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1.-disc_fake))

#Build optimizers
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [33]:
#Generator Variables
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]

#Discriminator Variables
disc_vars = [weights['disc_hidden1'], weights['disc_out'],
             biases['disc_hidden1'], biases['disc_out']]

#Create training operations
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)

#Init the variables
init = tf.global_variables_initializer()

In [43]:
#Start training
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(1, num_steps+1):
        #Prepare data
        #Get the next batch of data
        batch_x = next_batch(batch_size)
        #Generate noise
        z = np.random .uniform(-1., 1., size=[batch_size, noise_dim])
        
        #Train
        feed_dict = {disc_input: batch_x, gen_input: z}
        _, _, gl, dl = sess.run([train_gen, train_disc, gen_loss, disc_loss],
                               feed_dict=feed_dict)
        if i % 1000 == 0 or i == 1:
            print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))
    z = np.random.uniform(-1., 1., size=[4, noise_dim])
    g = sess.run([gen_sample], feed_dict={gen_input: z})
    print(g)

Step 1: Generator Loss: 0.603715, Discriminator Loss: 1.401730
Step 1000: Generator Loss: 2.055157, Discriminator Loss: 0.145031
Step 2000: Generator Loss: 4.028773, Discriminator Loss: 0.020532
Step 3000: Generator Loss: 5.147809, Discriminator Loss: 0.008191
Step 4000: Generator Loss: 6.629990, Discriminator Loss: 0.001562
Step 5000: Generator Loss: 7.652812, Discriminator Loss: 0.002592
Step 6000: Generator Loss: 6.994445, Discriminator Loss: 0.001090
Step 7000: Generator Loss: 8.699587, Discriminator Loss: 0.000228
Step 8000: Generator Loss: 8.725765, Discriminator Loss: 0.000180
Step 9000: Generator Loss: 9.647922, Discriminator Loss: 0.000679
Step 10000: Generator Loss: 10.114403, Discriminator Loss: 0.000042
Step 11000: Generator Loss: 8.557441, Discriminator Loss: 0.000225
Step 12000: Generator Loss: 9.775224, Discriminator Loss: 0.000062
Step 13000: Generator Loss: 9.052563, Discriminator Loss: 0.000126
Step 14000: Generator Loss: 11.353951, Discriminator Loss: 0.000014
Step 1